In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
import os
import time
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1500)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
tf.__version__

'2.0.0-beta1'

In [2]:
# Download the Shakespeare dataset

In [1]:
set('''You are all resolved rather to die than to famish?''')

{' ',
 '?',
 'Y',
 'a',
 'd',
 'e',
 'f',
 'h',
 'i',
 'l',
 'm',
 'n',
 'o',
 'r',
 's',
 't',
 'u',
 'v'}

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# Read the data
# First, look in the text:

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [7]:
# Process the text
# Vectorize the text
# # Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
# Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to len(unique).

In [10]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ',' :   6,
  'W' :  35,
  'F' :  18,
  'b' :  40,
  'o' :  53,
  'L' :  24,
  'n' :  52,
  'e' :  43,
  'G' :  19,
  'i' :  47,
  'H' :  20,
  'g' :  45,
  't' :  58,
  'I' :  21,
  'z' :  64,
  'S' :  31,
  "'" :   5,
  'd' :  42,
  'f' :  44,
  's' :  57,
  ...
}


In [11]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [12]:
# The prediction task
# Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

# Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

In [13]:
# Create training examples and targets
# Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

# For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

# So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

# To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [14]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [15]:
# The batch method lets us easily convert these individual characters to sequences of the desired size.

In [16]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [17]:
# For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [18]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [19]:
# Print the first examples input and target values:

In [20]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [21]:
# Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character.

In [22]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [24]:
# Create training batches
# # We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [25]:
# Batch size
BATCH_SIZE = 16

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((16, 100), (16, 100)), types: (tf.int64, tf.int64)>

In [26]:
# Build The Model
# Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

# tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
# tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)
# tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [27]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [28]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [29]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [30]:
# For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

In [31]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(16, 100, 65) # (batch_size, sequence_length, vocab_size)


In [32]:
# In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (16, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (16, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (16, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [34]:
# To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

In [35]:
# Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

In [37]:
# Try it for the first example in the batch:

In [36]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [38]:
# This gives us, at each timestep, a prediction of the next character index:

In [39]:
# sampled_indices

In [40]:
# Decode these to see the text predicted by this untrained model:

In [41]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "e it; in the name o' the people\nAnd in the power of us the tribunes, we,\nEven from this instant, ban"

Next Char Predictions: 
 "U'3hl$GI:T\nsYjYimRZpEHw&UuojF3$aLEummu,fyH;Oosf3-FaANZxZrDYZL3:,$$.ZRIiuIqRbAs\nvgsgTOnGzemy'qcjITmuw"


In [42]:
# Train the model

In [43]:
# At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

In [44]:
# Attach an optimizer, and a loss function
# The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

# Because our model returns logits, we need to set the from_logits flag.

In [45]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (16, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1747713


In [46]:
# Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [47]:
model.compile(optimizer='adam', loss=loss)

In [48]:
# Configure checkpoints
# Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [49]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [50]:
# Execute the training
# To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [51]:
EPOCHS=3

In [52]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/3
690/690 [==============================] - 88s 127ms/step - loss: 2.0032
Epoch 2/3
690/690 [==============================] - 86s 124ms/step - loss: 1.4882
Epoch 3/3
690/690 [==============================] - 85s 123ms/step - loss: 1.3584


In [ ]:
# Generate text
# Restore the latest checkpoint
# To keep this prediction step simple, use a batch size of 1.

# Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

# To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [53]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_3'

In [54]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [55]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [56]:
# The prediction loop
# The following code block generates the text:

# It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

# Get the prediction distribution of the next character using the start string and the RNN state.

# Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

# The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.

In [57]:
# To generate text the model's output is fed back to the input

# Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [58]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [67]:
print(generate_text(model, start_string=u"KING: "))

InvalidArgumentError: Invalid input_h shape: [1,16,1024] [1,1,1024] [Op:CudnnRNN]

In [60]:
# The easiest thing you can do to improve the results it to train it for longer (try EPOCHS=30).

# You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

In [62]:
# Advanced: Customized Training
# The above training procedure is simple, but does not give you much control.

# So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement curriculum learning to help stabilize the model's open-loop output.

# We will use tf.GradientTape to track the gradients. You can learn more about this approach by reading the eager execution guide.

# The procedure works as follows:

# First, initialize the RNN state. We do this by calling the tf.keras.Model.reset_states method.

# Next, iterate over the dataset (batch by batch) and calculate the predictions associated with each.

# Open a tf.GradientTape, and calculate the predictions and loss in that context.

# Calculate the gradients of the loss with respect to the model variables using the tf.GradientTape.grads method.

# Finally, take a step downwards by using the optimizer's tf.train.Optimizer.apply_gradients method.

In [63]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [64]:
optimizer = tf.keras.optimizers.Adam()

In [65]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [66]:
# Training step
EPOCHS = 3

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

W0725 03:08:45.579073 139812483114752 util.py:244] Unresolved object in checkpoint: (root).optimizer
W0725 03:08:45.582937 139812483114752 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0725 03:08:45.583806 139812483114752 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0725 03:08:45.585247 139812483114752 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0725 03:08:45.585952 139812483114752 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0725 03:08:45.586794 139812483114752 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0725 03:08:45.587651 139812483114752 util.py:244] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.embeddings
W0725 03:08:45.588405 139812483114752 util.py:244] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-2.kernel
W0725 03:08:45.590414 139812483114752 util.py:2

Epoch 1 Batch 0 Loss 4.174720287322998
Epoch 1 Batch 100 Loss 2.423362970352173
Epoch 1 Batch 200 Loss 1.9913874864578247
Epoch 1 Batch 300 Loss 1.9547773599624634
Epoch 1 Batch 400 Loss 1.796228289604187
Epoch 1 Batch 500 Loss 1.6714733839035034
Epoch 1 Batch 600 Loss 1.6259645223617554
Epoch 1 Loss 1.6034
Time taken for 1 epoch 87.16478610038757 sec

Epoch 2 Batch 0 Loss 2.3900704383850098
Epoch 2 Batch 100 Loss 1.6024212837219238
Epoch 2 Batch 200 Loss 1.4591652154922485
Epoch 2 Batch 300 Loss 1.5200886726379395
Epoch 2 Batch 400 Loss 1.4618556499481201
Epoch 2 Batch 500 Loss 1.410273551940918
Epoch 2 Batch 600 Loss 1.4026905298233032
Epoch 2 Loss 1.4365
Time taken for 1 epoch 83.21971154212952 sec

Epoch 3 Batch 0 Loss 1.5247793197631836
Epoch 3 Batch 100 Loss 1.3949743509292603
Epoch 3 Batch 200 Loss 1.2890568971633911
Epoch 3 Batch 300 Loss 1.393104076385498
Epoch 3 Batch 400 Loss 1.3499655723571777
Epoch 3 Batch 500 Loss 1.3158984184265137
Epoch 3 Batch 600 Loss 1.31696462631225